<div align=right>
<img src="img/logosmall.png" width="100px" align=right>
</div>

There are many ways to slice the various underlying programming paradigms that programming languages are based on, but one particularly pervasive distinction (with very deep underlying theoretical roots) is that of *imperative languages* (that see computation as a list of instructions) vs. *functional* languages (that see computation as recursive function calls).

We won't go into the theoretical details here, except to say that Python is a *pragmatic* language that borrows (or steals) from multiple paradigms.  In fact, for a language that appears at first blush to be largely imperative, Python supports a number of functional constructs.  And not only does it support them, but it makes using them feel extremely idiomatic and "Pythonic".

In this section, we'll look at some functional constructs in Python.  In the next section, we'll look at object-oriented constructs — an entirely different paradigm usually built on top of an imperative language.

# Computing with lists

Lists are a basic data structure in most functional languages, and computation *using* lists is a very common functional paradigm.  Let's look at some of the more common operations, and how they're done in Python:

## Filtering

Remember the example we had in the *Conditions* Notebook where we *filtered* a list to of accessions retrieve all the accessions starting in `a`?

In [ ]:
accs = ['ab56', 'bh84', 'hv76', 'ay93', 'ap97', 'bd72']
starts_with_a = []

for accession in accs:
    if accession.startswith('a'):
        starts_with_a.append(accession)

print(starts_with_a)

Filtering a list in this fashion is an incredibly common operation, and Python has a built-in function `filter()`.  It takes two arguments:

* a function that returns a boolean value (i.e. a predicate function)
* an iterable that can be iterated over

It returns a genrator that, when iterated over, returns elments of the original function that satisfy the boolean function:

In [ ]:
# We build our own predicate function...
def starts_with_a(s):
    return s.startswith('a')

# ...then use it to filter the list of accessions:
list(filter(starts_with_a, accs))

>Note how we pass a function around, just like any other object.  In this case, we pass it as an argument to the `filter()` function.

>Being able to treat functions as "first class" objects — i.e. objects that can be treated like any other — makes for a language that's well suited to programming in a functional paradigm.

In case it's not entirely clear what `filter()` does, we can write our own version of it that works only on lists (and returns a list):

In [ ]:
def my_filter(function, a_list):
    result_list = []
    for element in a_list:
        if function(element):
            result_list.append(element)
    return result_list

my_filter(starts_with_a, accs)

>Here we've created *our own* function that takes another function as argument.

Another exmample of using `filter`:

In [ ]:
def is_positive(x):
    return x >= 0

numbers = [12, 18, -23, 0, -7, 125, 13, -31]

list(filter(is_positive, numbers))

And another:

In [ ]:
def is_fasta_header(s):
    return s.startswith('>')

%cd files

lines = open("sample1.fa", 'r').readlines()
list(filter(is_fasta_header, lines))

OK, it's a bit tedious to write those little one-line boolean functions (or *predicate functions*) all the time, especially when they only get used for one thing.

We can use a literal string or a literal number anywhere in our Python code without having to assign them a variable name.  Wouldn't it be nice if we could use a "literal function" as well without having to assign it a name in the current namespace using `def`?

It turns out that, for one-line functions anyway, we can — by using Python's `lambda` keyword. 

In [ ]:
(lambda x: x * x)(5)

>Literal `lambda` functions only need to be surrounded by parentheses when required to avoid ambiguity, as in the example above.

Without further ado, here are the previous three examples rewritten using `lambda`:

In [ ]:
list(filter(lambda x: x.startswith('a'), accs))

In [ ]:
list(filter(lambda x: x >= 0, numbers))

In [ ]:
list(filter(lambda x: x.startswith('>'), open("sample1.fa", 'r').readlines()))

I'll be honest, though:  lambda functions aren't very "fashionable" in Python these days.  For one thing, Python's `lambda` statement is ridiculously limited in power compared to similar constructs in other languages:  A `lambda` function is limited to one statement, and cannot have a code block.

Fear not!  Python has other constructs — similarly borrowed from functional languages — that makes statements like the above even *more* succinct, and definitely more readable.  We'll get to them in a minute…

## Mapping

The `map()` function is almost always mentioned in the same breath as `filter()`.  Whereas `filter()` filters a list based on a predicate function, `map()` maps a function over a list.

In other words, given a function `f()` and a list `l`, `map(f, l)` would apply the function `f()` to every element of the list `l` and return a new list with the results.

>Python's `map()` doesn't actually return a list, but a generator over which one can iterate.  As before we can  *coerce* the result of `map` to a list using the `list()` function.)

> In Python 2, `map()` returned a simmple list.

In [ ]:
def square(x):
    return x * x

print(numbers)
print(list(map(square, numbers)))

Or, more succinctly:

In [ ]:
list(map(lambda x: x * x, numbers))

In [ ]:
# Filter the Fasta headers from a list of lines in a Fasta file
headers = list(filter(lambda x: x.startswith('>'), open("sample1.fa", 'r').readlines()))

# Strip leading '>' and trailing '\n' from list of headers
headers = list(map(lambda x: x.strip('\n>'), headers))

print(headers)

Let's look at two more built-in functions that compute using entire lists — `any()` and `all()`.  These are so-called *reduction* functions, which take a list as argument and "reduce it" to a simple value (in the case of `any()` and `all()`, a boolean value):

`any()` and `all()` are both called with a list as argument.

* `any(L)` returns `True` if **at least one** element of `L` is truthy, else  `False`

* `all(L)` returns `True` if **every** element of `L` is truthy, else `False`

In [ ]:
list1 = [3, 6, -3, -5, 4, -8]
list2 = [8, 2, 19, 25, 36, 11]
list3 = [-5, -2, -118, -13, -21, -11]

# A simple predicate function
def is_positive(x):
    return x >= 0

for a_list in list1, list2, list3:
    predicate_list = list(map(is_positive, a_list))
    print(a_list)
    print("Element positive:", predicate_list)
    print("Any positive:    ", any(predicate_list))
    print("All positive:    ", all(predicate_list))
    print()

When we cover *comprehensions* later on, we'll see another compact format for expressions mapping and filtering operations that does not require ad hoc predicate functions *or* lambda functions.

# Generators

We now come to *generators* — a fairly advanced piece of Python lore.  Don't feel you have to understand everything in this section the first time through!

## Generator objects

We've seen several built-in functions (and methods) that return a *generator object* — something that *yields* values (items, objects) when you iterate over it.  An example was `map()`:

In [ ]:
generator_object = map(square, range(1, 11))

generator_object

Let's play a bit with the generator object.  For one thing, a generator can be *exhausted*:  We can iterate over it until it stops yielding values, and at that point it yields no further values:

In [ ]:
# implicitly iterate over the generator object by using list()
print(list(generator_object))

# try to do it again
print(list(generator_object))

By the time we try to iterate over the generator a second time, it is already exhausted and yields nothing further, so the result of the `list()` function call is an empty list.

Let's make a new generator object that will yield only four values before being exhausted:

In [ ]:
generator_object = map(square, range(1, 5))

Instead of iterating over the generator and allowing it to yield all values until exhausted, we can make the generator object yield only a single value by using the built-in function `next()`:

In [ ]:
print(next(generator_object))

Let's do this three more times:

In [ ]:
print(next(generator_object))
print(next(generator_object))
print(next(generator_object))

Our generator should now be exhausted.  Let's try to prompt it once more:

In [ ]:
print(next(generator_object))

It raises the `StopIteration` exception.  In fact, `StopIteration` is *implicitly* caught by Python's iteration system.  If we were to iterate over a generator (using, say, a `for` loop) we won't see the `StopIteration`, since the iteration system will catch the exception and use it as a signal that the generator is exhausted, and that code execution can therefore exit the `for` loop:

In [ ]:
generator_object = map(square, range(1, 5))
for i in generator_object:
    print(i, end=' ')

## Generator functions

Sometimes we want to create our own generator objects manually, and for this we can use *generator functions*.  Generator functions are a powerful and advanced feature of Python, and we'll only cover the very basics here.

Generator functions are defined (like normal) functions using the keyword `def`.  Whereas a normal function may (or may not) return a value using the `return` keyword, a generator function **always** contains a `yield` statement.

>This is worth repeating since it can lead to a lot of confusion, since Python's syntax unfortunately doesn't make the distinction between a function and a generator function very clear:

>A function definition that contains the keyword `yield` is **always** a generator function, which is something quite different from a normal function.

When you *call* a generator function it doesn't get executed like a normal Python function; instead it yields a generator object which can be iterated over to yield results.  You specify how those results are to be yielded using the `yield` keyword.

Here's a generator function that, when called, will return a generator which will yield all fibonacci numbers smaller than 10000 when iterated over:

In [ ]:
def fib_10k():
    a, b = 1, 1
    while a < 10000:
        yield a
        a, b = b, a + b

Let's execute `fib_10k` and assign whatever it returns to a variable `g`, then inspect that thing:

In [ ]:
g = fib_10k()

g

You can clearly see that `g` is a generator object.  Let's iterate over it and see what it yields:

In [ ]:
for i in g:
    print(i)

Let's implicitly iterate over our generator `g` using the `list` function:

In [ ]:
list(g)

Oops.  The generator object referenced by the variable `g` has been exhausted.  We've already iterated over it all the way "to the end", so it yields no further values.

However, our generator function can provide us with as many generators as we wish!

In [ ]:
g = fib_10k()
print(list(g))

Once again we can "manually" advance a generator by using the built-in function `next()`:

In [ ]:
g = fib_10k()

print(next(g))
print(next(g))
print(next(g))

print(list(g))

Note that the generator `h` "remembered its state".  It had already yielded the first three fibonacci numbers (1, 1, and 2), so when we coerced it to a list with `list()`, it continued on from there.

>One way of looking at a generator is as "a function that remembers state".  It's not entirely accurate, but it's sometimes useful to think of it like this.

Note that generators are inherently *lazy* — they only yield values when they are called upon to do so.  Thus, one can get generators to fill the role of things that aren't normally possible in a *strict* programming langauge.  (*strict* and *lazy* are opposites to a computer scientist.)

An infinite list is an example of something that can only be created as a lazy construct.  The fibonacci sequence has no upper bound, and one could easily create a generator that will yield an infinite sequence of fibonacci numbers:

In [ ]:
def fib():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b

If you were to execute that generator function, and then try to iterate over the resulting generator, it would simply keep going until your computer's memory has been exhausted.  **DON'T DO IT.**

Generator functions can of course take arguments as well; in fact, they mostly do.  Let's create a generator function that yields all fibonacci numbers up to `n`:

In [ ]:
def fib(n):
    "Yield all fibonacci numbers up to 'n'"
    a, b = 1, 1
    while a <= n:
        yield a
        a, b = b, a + b

fib_100 = fib(100)
fib_1000 = fib(1000)

print(list(fib_100))
print(list(fib_1000))

It's perfectly permissible to have multiple `yield` statements in a generator function.

In [ ]:
from math import pi, e

def important_number_maker(your_number):
    yield pi
    yield your_number
    yield e
    
num_gen = important_number_maker(42)
list(num_gen)

We can manually tell a generator when to finish by explicitly raising `StopIteration` in our generator function.

Here's a function that takes two arguments, `upper_limit` and `stop_limit`.  It will return a generator that yields random integers between 1 and `upper_limit` until it yields a number greater than `stop_limit`, at which point the iteration will end.

In [ ]:
import random

def randoms(upper_limit, stop_limit):
    while True:
        result = random.randint(1, upper_limit)
        yield result
        if result > stop_limit:
            raise StopIteration

rgen = randoms(100, 90)
print(list(rgen))

Run the above code cell multiple times (use `Ctrl-Enter` to run a code cell without advancing the cursor) and see how the resulting list differs every time.

# Comprehensions

Now we come to one of my favourite Python constructs:  Comprehensions!

Strictly speaking, comprehensions are *syntactic sugar*.  In other words, they don't provide any functionality we cannot achieve with other, more basic syntactic elements in Python.  Syntactic sugar doesn't always have the best of reputations…

<div class="alert alert-info">
Syntactic sugar causes cancer of the semicolon.

<div align="right">
— Alan J Perlis, "Epigrams on Programming"

But comprehensions provide such a wonderfully clean syntax for mapping and filtering operations on lists (and other iterables) that it's hard not to love them!  After all, a syntax that's both concise and very readable can make your code shorter *and* easier to understand.

>For the math nerds:  Comprehensions are based on the Zermelo-Fraenkel set notation.

When we learned about the `map()` function earlier, we used it to create a list of squares:

In [ ]:
numbers = [12, 18, -23, 0, -7, 125, 13, -31]

def squares(x):
    return x * x

list(map(squares, numbers))

We then restated this even more succinctly using a lambda function:

In [ ]:
list(map(lambda x: x * x, numbers))

That's pretty short and sweet, but many people find it a little hard to read, and even unintuitive.

If we had used a *list comprehension* to create our list of squares, we would have done it like this:

In [ ]:
[x * x for x in numbers]

Now I think most people would agree that is both short **and** clearly readable!

Note that the list comprehension yielded an actual list, not a generator object like `map()`.  Sometimes (often!) it's more efficient to work with a generator object than a list, and there's a kind of comprehension that creates generators as well, as we'll soon see.

## List comprehensions

A *list comprehension* is a statement in Python that evaluates to a list.  It can have the form:

```python
[<statement> for <variable> in <iterable>]
```

…where `<variable>` is available for use in `<statement>`.
    
This is roughly equivalent to:

```python
result = []
for <variable> in <iterable>:
  result.append(<statement>)
result
```

It's actually much easier to understand based on examples, rather than trying to define it explicitly!

In [ ]:
# Grab the first 5 characters from every line in a file

[line[:5] for line in open("10_sequences.txt", 'r')]

In [ ]:
# Get a list of value/key (not key/value) tuples from a dict

counts = {'CGC': 1, 'ACG': 1, 'CGA': 1, 'CGT': 1, 'TAC': 1,
          'ATC': 2, 'TGA': 2, 'CTG': 1, 'GTA': 1, 'ATG': 1,
          'AAT': 1, 'GAT': 2, 'TCG': 2, 'GCT': 1}

[(value, key) for key, value in counts.items()]

Note in the example above that tuple unpacking is alive and well in list comprehensions.

We can map a predicate over a list using a comprehension:

In [ ]:
[n >= 0 for n in numbers]

We could use a list comprehension to convert a line from a CSV file into a list of integers.  The comprehension itself turns the list of strings returned by the `split()` method int a list of integers by mapping the `int()` function over it:

In [ ]:
csv = "12,67,13,129,78,23"
[int(i) for i in csv.split(',')]

…though this is also achievable in very readable form by using `map()`.  It's really a matter of personal preference in this case:

In [ ]:
csv_numbers = map(int, csv.split(','))
list(csv_numbers)

All of this, however, was only the first half of the syntax of list comprehensions.  Just as list comprehensions can perform **mapping** operations, they can also perform **filtering** operations.  In fact, they're at their best when they do both at the same time!

Recall how earlier (much earlier!) we filtered a list of accessions, keeping only those starting in 'a'.  We first did it by iterating over the list, and building up the resulting filtered list manually:

In [ ]:
accs = ['ab56', 'bh84', 'hv76', 'ay93', 'ap97', 'bd72']

starts_with_a = []
for accession in accs:
    if accession.startswith('a'):
        starts_with_a.append(accession)
        
print(starts_with_a)

Later we used the `filter()` function to restate this more succinctly:

In [ ]:
def starts_with_a(s):
    return s.startswith('a')

list(filter(starts_with_a, accs))

If we had filtered the list using a list comprehension, we could have achieved the same thing in an even shorter *and* more readable format:

In [ ]:
[a for a in accs if a.startswith('a')]

Adding the mapping functionality to list comprehensions, we now have the syntax:

```python
[<statement> for <variable> in <iterable> if <condition>]
```

…where `<variable>` is available for use in `<statement>` and `<condition>`.

The `for` clause is required;  the `if` clause is optional.
    
This is roughly equivalent to:

```python
result = []
for <variable> in <iterable>:
    if <condition>
        result.append(<statement>)
result
```

Note that a single list comprehension can perform *both* mapping and filtering:

In [ ]:
[x * x for x in numbers if x >= 0]

## Generator comprehensions

From list comprehensions it's just a skip and a jump to *generator comprehensions*.

A generator comprehension is nearly identical to a list comprehension, but instead of a literal list, it yields a generator object.  If you iterate over this generator object, it yields the elements of the equivalent list comprehension.

Syntactically, the only difference between liste comprehensions and generator comprehensions is that the former is written in square brackets, (“`[]`”), and the latter in parentheses (“`()`”).

In [ ]:
squares_list = [x * x for x in numbers]
squares_gen = (x * x for x in numbers)

print("list:     ", squares_list)
print("generator:", squares_gen)

As we know by now, we can *coerce* the generator to a list by using `list()`:

In [ ]:
list(squares_gen)

The defining characteristic of a generator, as before, is the fact that it's *lazy*.  If write the following statement, it creates a generator which will read in a file and yield the first five characters of every line of that file.  It then assigns the variable `leading_4` to this generator:

In [ ]:
leading_5 = (line[:5] for line in open("10_sequences.txt", 'r'))

It's very important to note that the file `10_sequences.txt` has *not actually bee read into memory immediately* as it would have been if I had used a list comprehension.

At this point, right now, the file has been opened but no data has been read from it yet.

If I iterate over the generator `leading_5`, the file will be read into memory *one line at a time*, and the first five characters from each line will be yielded.

At *no point* will the entire file be in the computer's memory.  Obviously this can be very important when we're dealing with huge data files.

In [ ]:
for line_start in leading_5:
    print("First 5 chars:", line_start)

Generator comprehensions have another useful bit of associated syntactic sugar:  If a generator comprehension is the *only* argument to a function, we may leave out its enclosing parentheses.

For example, the built-in function `sum()` takes a list (or other iterable) as argument, and returns the sum of the elements of that list (or iterable):

In [ ]:
sum([x * x for x in numbers])

We could also have computed the sum of the squares using a generator comprehension instead of a list comprehension:

In [ ]:
sum((x * x for x in numbers))

…and if we did that, we could have ommitted the inner pair of parentheses, leaving this very clear syntax:

In [ ]:
sum(x * x for x in numbers)

Another example, testing whether any of the elements of `numbers` is positive:

In [ ]:
any(x >= 0 for x in numbers)

Here's one way to invert the keys and values of a dictionary, and create a new dictionary out of that:

In [ ]:
inv_counts = dict((value, key) for key, value in counts.items())

…though this isn't a smart thing to do if the original dictionary has duplicate values (as `counts` does!)  It works, but we overwrite items as we go along, leading to a truncated dictionary:

In [ ]:
inv_counts

>In general, you can't invert a dictionary to create another dictionary unless you *know* for a fact that the values are unique.  (In the Python standard library there are advanced data structures, like dictionary-like objects that don't require unique keys.)

Here's a nice, slightly more complex example:  Let's use generator comprehensions to write a function that calculates all the prime numbers up to and including an arbitrary integer `k`.

First we introduce **two** new mathematical operators that's built into the core Python language.

The *modulo* operator written as “`%`”.  In mathematics, "a modulo b" is the *remainder* if a is divided by b:

In [ ]:
27 % 5

The *floor division* operator is written as “`//`”.  Floor division yields the integer part of a quotient, with the remainder discarded:

In [ ]:
27 // 5

Note that using "normal" division, the same statement would of course have yielded a floating point number:

In [ ]:
27 / 5

>In Python 2, the standard division operator “`/`” yields an integer quotient if the dividend and divisor are both integers.  In other words, it always performs floor division.  This can easily lead to subtle bugs!

Now:

* a number `n` is prime if if has no factors


* `i` is a factor of `n` if `n % i == 0`

To test whether `n` is prime, we have to test whether `n % i` is non-zero for all `i` in the range from 2 to `n//2`.  (Obviously, no number larger than `n//2` will be a factor of `n`.)

In [ ]:
def is_prime(n):
    return all(n % i for i in range(2, n // 2))

for p in [2, 3, 8, 11, 17, 21, 23, 28, 29, 31]:
    if is_prime(p):
        print("{} is prime".format(p))
    else:
        print("{} is not prime".format(p))

Finally, our function to return all primes smaller than or equal to `k`:

In [ ]:
def primes(k):
    return (i for i in range(2, k + 1) if is_prime(i))

print(list(primes(500)))

We still haven't seen the end of the comprehensions syntax:  Comprehensions can have multiple `for` clauses, **and** multiple `if` clauses.  Arguably, a comprehension with too many clauses can become harder to read than the equivalent nested `for` loops and `if` statements, so we won't explicitly look at any examples of such complex comprehensions.

Well, maybe just one:  Here is a list comprehension to obtain a list of all possible trinucleotides:

In [ ]:
trinucs = [s + t + u
           for s in "ACTG"
           for t in "ACTG"
           for u in "ACTG"
          ]

print(trinucs)

>Note that Python's rule that we can break a line on whitespace within brackets or parentheses also holds for comprehensions.

---

# Exercises

### Exercise 1

Given two positive integers `a` and `b`, calculate the sum of all the odd integers from `a` to `b` (inclusive).

> Hint: You'll need the modulo operator `%`, and using a comprehension can yield a very elegant solution.

Test yourself:  If `a` is 100 and `b` is 200, the result is 7500.

In [ ]:
# Exercise 1

def sum_odds(a, b):
    return sum(i for i in range(a, b + 1) if i % 2)

### Exercise 2

>This example is taken from Project Rosalind.  The original can be found here:

>* <http://rosalind.info/problems/hamm/>

Given two strings `s` and `t` of equal length, the Hamming distance between `s` and `t` is the number of corresponding symbols that differ in `s` and `t`:

![Hamming distance](img/hamming.png)

In the `files` subdirectory lies a file called `hamm.txt`.  It contains two sequences — one per line — of 977 bases each.

Return: The Hamming distance between these two sequences.

>Hint: This problem can be solved quite elegantly using comprehensions.

In [ ]:
# Exercise 2